In [ ]:
# To store the data
import pandas as pd

# To do linear algebra
import numpy as np

# To create plots
import matplotlib.pyplot as plt
import seaborn as sns

# To create interactive plots
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# To shift lists
from collections import deque

# To compute similarities between vectors
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# To use recommender systems
#import surprise as sp
#from surprise.model_selection import cross_validate

# To create deep learning models
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model

# To create sparse matrices
from scipy.sparse import coo_matrix

# To light fm
#from lightfm import LightFM
#from lightfm.evaluation import precision_at_k

# To stack sparse matrices
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

#from fuzzywuzzy import fuzz

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
# Get Data
new_Combined = 'C:/Users/passi/OneDrive - Hochschule Osnabrück/Semester 5-DESKTOP-P9SBT98/Programmierprojekt/Combined_Data_All.gzip'
movie_tile_File = 'C:/Users/passi/OneDrive - Hochschule Osnabrück/Semester 5-DESKTOP-P9SBT98/Programmierprojekt/movie_titles.csv'

data = pd.read_parquet(new_Combined)
Convert_dic_data= {'Cust_Id': 'int64', 'Rating': 'float32', 'Movie_Id': 'Int64'}
data= data.astype(Convert_dic_data)
del new_Combined

movie_titles = pd.read_csv(movie_tile_File,
                           encoding = "ISO-8859-1",
                           delimiter= '\t',
                           header = None,
                           names = ['Target'])
                           # Speicher alle Daten erst in eine Reihe, danach trennt er diese
movie_titles[['Movie_Id', 'Year', 'Name']] = movie_titles['Target'].str.split(pat=",",n=2, expand=True)   
movie_titles= movie_titles.drop(['Target', 'Year'], axis= 1)  
Convert_dic= {'Movie_Id': 'int64', 'Name': 'str'}
movie_titles = movie_titles.astype(Convert_dic)
#movie_titles.astype({'Movie_Id': 'int64', 'Name': 'str'}).dtypes
#movie_titles.rename(columns={'Id':'Movie_Id'}, inplace=True)
#movie_titles = movie_titles.reset_index(drop=True)
display(movie_titles)

del Convert_dic


####################################################
# Filter Data
#

df = data
# Filter sparse movies
min_movie_ratings = 1000
filter_movies = (df['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 200
filter_users = (df['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df[(df['Movie_Id'].isin(filter_movies)) & (df['Cust_Id'].isin(filter_users))]
del filter_movies, filter_users, min_movie_ratings, min_user_ratings, data, df


print('Shape User-Ratings unfiltered:\t{}'.format(df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))

In [76]:
# pivot and create movie-user matrix

movie_user_mat = df_filterd.pivot(index='Movie_Id', columns='Cust_Id', values='Rating').fillna(0)

#print(movie_user_mat[i])
print(movie_user_mat.loc[movie_user_mat['Cust_Id'] == 3])

In [12]:
user_movie_table_matrix = csr_matrix(movie_user_mat.values)
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'auto',n_jobs=-1)
model_knn.fit(user_movie_table_matrix)

NearestNeighbors(metric='cosine', n_jobs=-1)

In [ ]:
# Random Film wird ausgewählt!
#query_index = movie_user_mat.loc[movie_user_mat['movie_id'] == 3]
print(movie_user_mat.loc[[11977]].values.reshape(1,-1), n_neighbors = 120)

Choosen Movie is:  14327


In [62]:
# Berechnen des Nächsten Nachbarn mit Random Film
distances, indices = model_knn.kneighbors(movie_user_mat.iloc[movie_user_mat.loc[movie_user_mat['movie_id'] == id],:].values.reshape(1,-1), n_neighbors = 120)
#Fehler wenn der Index 0 ist (Index von 1 - 7124)

In [ ]:
## Ausgabe
import re


movie = []
distance = []

for i in range(0, len(distances.flatten())):
    if i != 0:
        movie.append(movie_user_mat.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])    

m=pd.Series(movie,name='Movie_Id')
d=pd.Series(distance,name='distance')
recommend = pd.concat([m,d], axis=1)
recommend = recommend.sort_values('distance',ascending=False)

print('Recommendations for {0}:\n'.format(movie_user_mat.index[query_index]))
recMovies = recommend["Movie_Id"].iloc[range(0,5)]
#for i in range(0,5): # Für die ersten 5 Filme 
    #print('{0}: {1}, with distance of {2}'.format(i, recommend["Movie_Id"].iloc[i], recommend["distance"].iloc[i]))

    #recMovies (recommend["Movie_Id"].iloc[i])

print(recMovies)
###